### Code definitions

In [1]:
import os
import sys
from pathlib import Path
from asgiref.sync import sync_to_async
import string
import pandas as  pd
from datetime import date
import yfinance as yf
import sqlite3

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'share_dinkum_proj.settings')

import django
django.setup()

from django.conf import settings
import share_dinkum_app.models as app_models

from share_dinkum_app import loading

In [2]:
@sync_to_async
def add_default_account(
    default_name='admin',
    default_email='admin@example.com',
    password='password',
    default_portfolio_description = 'Default Portfolio'
    ):
    
    fiscal_year_type, created = app_models.FiscalYearType.objects.get_or_create(
        description='Australian Tax Year',
        defaults={'start_month': 7, 'start_day': 1}
    )

    if not app_models.AppUser.objects.filter(username=default_name).exists():
        user = app_models.AppUser.objects.create_superuser(username=default_name, email=default_email, password=password)   # Please change this password after logging in
        print("Superuser created successfully!")
    owner = app_models.AppUser.objects.get(username=default_name)

    record = {'description': default_portfolio_description, 'owner': owner, 'fiscal_year_type': fiscal_year_type}
    account, created = app_models.Account.objects.get_or_create(description=default_portfolio_description, defaults=record)
    return account

@sync_to_async
def clear_all_data():
    loading.clear_all_data()

@sync_to_async
def load_all_data(account, input_file):
    loading.DataLoader(account=account, input_file=input_file)

@sync_to_async
def load_historical_prices(df, account):
    loader = loading.DataLoader(account=account, input_file=None)
    loader.load_table_to_model(model=app_models.InstrumentPriceHistory, df=df)
    
@sync_to_async
def update_all_price_history(account):
    account.update_all_price_history()

In [3]:
await clear_all_data()

Error deleting model Rate: no such table: djmoney_rate
Error deleting model ExchangeBackend: no such table: djmoney_exchangebackend
Deleted all models
Forcefully deleted and recreated folder: C:\code\share-dinkum\share_dinkum_proj\media


In [4]:
account = await add_default_account()
account

Superuser created successfully!


<Account: Default Portfolio | AUD>

In [5]:
import_data_folder = settings.BASE_DIR / 'share_dinkum_app' / 'import_data'

personal_data = import_data_folder / 'data_import_template_personal.xlsx'
generic_data = import_data_folder / 'data_import_template_public.xlsx'


if personal_data.exists():
    print('Using own data')
    input_file = personal_data
else:
    print('Using generic data')
    input_file = generic_data

input_file


Using own data


WindowsPath('C:/code/share-dinkum/share_dinkum_proj/share_dinkum_app/import_data/data_import_template_personal.xlsx')

In [6]:
await load_all_data(account=account, input_file=input_file)

Loading Market


11it [00:00, 70.93it/s]


Loading Instrument


45it [00:01, 36.55it/s]


Loading Buy


...

40it [00:03, 13.06it/s]


Loading Dividend


296it [00:03, 90.45it/s]


Loading Distribution


180it [00:02, 89.90it/s]


In [7]:
# You can ignore this cell

legacy_db = Path(r'C:\code\finance-database\finance-database.db')

if legacy_db.exists():
    query = "SELECT * FROM [price_history] where ticker_code != 'AUDUSD=X'"
    parameters = None
    with sqlite3.connect(legacy_db) as conn:
        df = pd.read_sql_query(query, conn, params=parameters)
    df['account'] = account
    df['instrument__name'] = df['ticker_code'].apply(lambda x : x.split('.')[0])
    df['date'] = df['date'].apply(lambda x : date.fromisoformat(x))
    df = df.drop(columns=['id', 'ticker_code', 'capital_gains', 'dividends'])
    df = df[~df['high'].isna()]
    await load_historical_prices(df=df, account=account)

161231it [10:38, 252.69it/s]


In [ ]:
await update_all_price_history(account=account)